# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/flaviolin/DBTL[.]/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/flaviolin/DBTL[.]/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [2]:
CYCLE = 2

user_params = {
    'media_file': f'../data/flaviolin/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../data/flaviolin/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../data/flaviolin/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 4,
    'num_designs': 12,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [3]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4
Well,,,,,,,,,,,,,,,
A1,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A2,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A3,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A4,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A5,40.0,4.0,0.001199,20.0,2.356433,0.965094,0.005574,10.252992,0.428695,133.073471,0.000090,0.000246,0.000706,0.002361,0.000076


## Create Line Description

In [4]:
df['Line Description'] = df.apply(create_media_description, axis=1)


## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]_W[..]-R[.]** denoting cycle number, wells occupying the same design and the replicate number:

In [5]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

lnfcn = lambda x: f'C{CYCLE}_W{x.name[0]}1_{x.name[0]}4-R{x["Replicate"]}' if int(x.name[1]) < 5 else f'C{CYCLE}_W{x.name[0]}5_{x.name[0]}8-R{x["Replicate"]}'
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(lnfcn, axis=1)

In [6]:
df.head(2)

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.00079,0.00056,0.000172,"MOPS: 40.0000, Tricine: 4.0000, H3BO3: 0.0270,...",1,C2_WA1_A4-R1
A2,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.00079,0.00056,0.000172,"MOPS: 40.0000, Tricine: 4.0000, H3BO3: 0.0270,...",2,C2_WA1_A4-R2


## Process measurement files

Read measurements file:

In [7]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.0467,0.0483,0.0526,0.0531,0.0434,0.0414,0.0433,0.0428,0.0355,0.0462,0.0462,0.0457
B,0.1290,0.1209,0.1126,0.1230,0.0588,0.0651,0.0521,0.0602,0.0470,0.0469,0.0465,0.0457
C,0.1297,0.1484,0.1360,0.1341,0.1467,0.1539,0.1437,0.1468,0.0470,0.0471,0.0468,0.0456
D,0.0620,0.0697,0.0647,0.0623,0.0698,0.0889,0.0906,0.1030,0.0469,0.0467,0.0463,0.0462
E,0.0743,0.0589,0.0757,0.0716,0.0736,0.0872,0.0699,0.0509,0.0465,0.0466,0.0463,0.0460
F,0.0459,0.0445,0.0451,0.0452,0.0986,0.0948,0.0956,0.1026,0.0466,0.0465,0.0462,0.0459
G,0.0462,0.0466,0.0472,0.0468,0.0467,0.0464,0.0462,0.0465,0.0467,0.0464,0.0461,0.0474
H,0.0462,0.0465,0.0458,0.0458,0.0470,0.0457,0.0456,0.0457,0.0461,0.0462,0.0459,0.0455


In [8]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.4943,0.4974,0.5088,0.4950,0.4947,0.4859,0.4936,0.4945,0.1226,0.1339,0.1320,0.1378
B,0.3844,0.3730,0.3769,0.3893,0.4937,0.4857,0.4862,0.4927,0.1383,0.1369,0.1344,0.1311
C,0.2998,0.3093,0.3153,0.3228,0.3945,0.3983,0.4009,0.4043,0.1374,0.1370,0.1355,0.1333
D,0.5300,0.5533,0.5545,0.5384,0.3037,0.3098,0.3057,0.3148,0.1380,0.1377,0.1361,0.1315
E,0.5768,0.5747,0.5876,0.6018,0.2451,0.2819,0.2774,0.2636,0.1379,0.1378,0.1359,0.1320
F,0.4254,0.4236,0.4232,0.4291,0.4160,0.4138,0.4230,0.4216,0.1383,0.1372,0.1349,0.1328
G,0.1301,0.1347,0.1373,0.1379,0.1385,0.1361,0.1362,0.1388,0.1380,0.1366,0.1343,0.1324
H,0.1308,0.1333,0.1339,0.1344,0.1339,0.1331,0.1330,0.1336,0.1345,0.1340,0.1328,0.1305


### Process OD600

Normalize the data to the control well A9 value (water content):

In [9]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.0112,0.0128,0.0171,0.0176,0.0079,0.0059,0.0078,0.0073,0.0000,0.0107,0.0107,0.0102
B,0.0935,0.0854,0.0771,0.0875,0.0233,0.0296,0.0166,0.0247,0.0115,0.0114,0.0110,0.0102
C,0.0942,0.1129,0.1005,0.0986,0.1112,0.1184,0.1082,0.1113,0.0115,0.0116,0.0113,0.0101
D,0.0265,0.0342,0.0292,0.0268,0.0343,0.0534,0.0551,0.0675,0.0114,0.0112,0.0108,0.0107
E,0.0388,0.0234,0.0402,0.0361,0.0381,0.0517,0.0344,0.0154,0.0110,0.0111,0.0108,0.0105
F,0.0104,0.0090,0.0096,0.0097,0.0631,0.0593,0.0601,0.0671,0.0111,0.0110,0.0107,0.0104
G,0.0107,0.0111,0.0117,0.0113,0.0112,0.0109,0.0107,0.0110,0.0112,0.0109,0.0106,0.0119
H,0.0107,0.0110,0.0103,0.0103,0.0115,0.0102,0.0101,0.0102,0.0106,0.0107,0.0104,0.0100


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [10]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,0.112,0.128,0.171,0.176,0.079,0.059,0.078,0.073
B,0.935,0.854,0.771,0.875,0.233,0.296,0.166,0.247
C,0.942,1.129,1.005,0.986,1.112,1.184,1.082,1.113
D,0.265,0.342,0.292,0.268,0.343,0.534,0.551,0.675
E,0.388,0.234,0.402,0.361,0.381,0.517,0.344,0.154
F,0.104,0.090,0.096,0.097,0.631,0.593,0.601,0.671


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [11]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.3717,0.3748,0.3862,0.3724,0.3721,0.3633,0.3710,0.3719
B,0.2618,0.2504,0.2543,0.2667,0.3711,0.3631,0.3636,0.3701
C,0.1772,0.1867,0.1927,0.2002,0.2719,0.2757,0.2783,0.2817
D,0.4074,0.4307,0.4319,0.4158,0.1811,0.1872,0.1831,0.1922
E,0.4542,0.4521,0.4650,0.4792,0.1225,0.1593,0.1548,0.1410
F,0.3028,0.3010,0.3006,0.3065,0.2934,0.2912,0.3004,0.2990


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [12]:
df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

indfcn = lambda x: f'{x["variable"]}{(x.name)}'

df_600['Well'] = df_600.apply(indfcn, axis=1)
df_600.index = df_600['Well']
df_600.drop(columns=['variable', 'Well'], inplace=True)

df_340['Well'] = df_340.apply(indfcn, axis=1)
df_340.index = df_340['Well']
df_340.drop(columns=['variable', 'Well'], inplace=True)

df_600.head()

,OD600
Well,
A1,0.112
A2,0.128
A3,0.171
A4,0.176
A5,0.079


Add measurements to the main dataframe:

In [13]:
measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name,OD600,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.00079,0.00056,0.000172,"MOPS: 40.0000, Tricine: 4.0000, H3BO3: 0.0270,...",1,C2_WA1_A4-R1,0.112,0.3717
A2,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.00079,0.00056,0.000172,"MOPS: 40.0000, Tricine: 4.0000, H3BO3: 0.0270,...",2,C2_WA1_A4-R2,0.128,0.3748


## Create EDD Experiment Description File

In [14]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [15]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [16]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [17]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
df['Measurement Type'] = 'Optical Density'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD600]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [18]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)